In [3]:
from ASU_env import ASUEnv
from ASU_opt_model import optimize_ASU
from copy import deepcopy
from pathlib import Path
import numpy as np

In [4]:
# base_dir   = Path(__file__).resolve().parent
base_dir = Path().resolve()
config_fp  = base_dir / "asuenv_config.json"
if not config_fp.is_file():
    raise FileNotFoundError(f"Couldn’t find config.json at {config_fp}")
lookahead = 4
env_id = 'ASU1'
env = ASUEnv('ASU1', config_path = config_fp)

# get lookahead value from the environment to feed into the optimizer
asu_optimizer = optimize_ASU('ASU1', lookahead)

env.reset()
current_day = 1
optimization_day1_cost = {}
opt_lambda = {}
opt_reward = {}
num_action = 0

terminated = False
while not terminated:

    # state = env.render()
    state = env._get_state(mode='dict')
    asu_optimizer.update_state(state)
    
    # Solve the optimization model
    objective = asu_optimizer.solve()
    day1_cost = asu_optimizer.day1_cost()
    optimization_day1_cost[current_day] = day1_cost  
    
    opt_day_lambda = asu_optimizer.extract_optimal_lambda()
    # fetch optimal hourly lambda values for the current day
    opt_lambda[current_day] = opt_day_lambda
    # based on opt_day_lambda, calculate the reward: opt_reward[current_day]
    
    day_reward = 0.0
    for hour in range(1, 25):
        # Retrieve the lambda values for this hour (assume keys in subdict are sorted correctly, e.g., 1, 2, ...).
        hourly_lambda = opt_day_lambda[hour]
        # Convert the lambda dict to a NumPy array. # For instance, if your extreme points indices are [1,2,...,n], sort the keys and build the array.
        lambda_array = np.array([hourly_lambda[k] for k in sorted(hourly_lambda.keys())], dtype=np.float32)
        # opt_action = {'lambda': lambda_array}
        opt_action = lambda_array
        
        # Take a step in the environment using this optimal action.
        _, reward, terminated, _ , _ = env.step(opt_action)
        num_action += 1
        terminated = terminated.item()
        day_reward += reward.item()

    opt_reward[current_day] = day_reward

    # Approaching end of current day
    if terminated == True:
        print("number of actions taken: {}".format(num_action))
        print("Simulation complete for total of {} days".format(current_day))
    current_day += 1
print(optimization_day1_cost)
print(opt_reward)

number of actions taken: 168
Simulation complete for total of 7 days
{1: 1044.5566829738332, 2: 861.3592639213889, 3: 805.6513215031631, 4: 1007.3574626449497, 5: 887.7661076585582, 6: 927.1291526642696, 7: 1063.684094797056}
{1: -1044.55668258667, 2: -861.3592681884766, 3: -805.6513214111328, 4: -1007.3574638366699, 5: -887.7660980224609, 6: -927.2235565185547, 7: -1063.684097290039}
